In [1]:
import pandas as pd
import datetime
import numpy as np
from pandarallel import pandarallel

pd.set_option('display.max_columns',None)

Your first task is to create a stacked chart of **unique online devices per
day**, segregated by fleet size. The fleet size is an attribute of each user
and is defined as the number of online devices that this user had at a
particular day. You can split the dataset in the following fleet sizes:

* 1-2 devices
* 3-9 devices
* 10-99 devices
* 100-999 devices

A device should be counted as online for a particular day if it was online for
any amount of time during that day. For example, a device that appear online
for only a second should still be counted for that day.

The specific rules we have selected to deal with problematic sections
of a device's timeline can be summarised in the following table:

| current event | current server | next event | next server | rule        |
|---------------|----------------|------------|-------------|-------------|
| online        | X              | online     | X           | Assume device was online from current event's timestamp until next event's timestamp
| online        | X              | online     | Y           | Assume device was online from current event's timestamp until X's destruction time or next event's timestamp, whichever is smaller
| online        | X              | offline    | X           | Normal case
| online        | X              | offline    | Y           | Assume device was online from current event's timestamp until X's destruction time. Ignore next event.
| offline       | X              | online     | X           | Normal case
| offline       | X              | online     | Y           | Normal case
| offline       | X              | offline    | X           | Ignore next event
| offline       | X              | offline    | Y           | Ignore next event

In [71]:
def prev_server(df):
    """
    calculate previous server connection's data: previous server id, previous connected status and previous timestamp
    """
    df['prev_server_id']=(df.sort_values(['timestamp'], ascending=True)
                               .groupby(['device_id'])['server_id'].shift(-1))
    df['prev_connected']=(df.sort_values(['timestamp'], ascending=True)
                             .groupby(['device_id'])['connected'].shift(-1)).astype(bool)
    df['prev_timestamp']=(df.sort_values(['timestamp'], ascending=True)
                                             .groupby(['device_id'])['timestamp'].shift(-1))
    df['ignored']=False

def mark_ignored_rows(row):
    """
    function to mark rows as ignored, cases:
    1. X offline -> X offline
    2. X offline -> Y offline
    """
    return row['connected']==False and row['prev_connected']==False
    
def process_dataframe(df):
    df['ignored'] = df.apply(lambda row: prepare_final(row), axis=1)

def parallelize_dataframe(df, func, n_cores=4):
    pool = Pool(n_cores)
    df_split = np.array_split(df, n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [8]:
names=['id','created_at','destroyed_at']
servers = pd.read_csv('data/servers.csv',names=names)

In [9]:
names = ['timestamp','device_id', 'user_id','server_id','connected']
events = pd.read_csv('data/connectivity_events.csv',names=names)#,nrows=1000)

In [23]:
events_merged=events.merge(servers,left_on='server_id',right_on='id')
print(events_merged.shape[0])
events_merged.head()

9308207


,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929


In [21]:
%%timeit
generate_previous_connection(events_merged)
events_merged.head()

26.2 s ± 436 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
events_merged.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected,prev_timestamp,ignored
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,NaN,True,NaN,False
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,NaN,True,NaN,False
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,NaN,True,NaN,False
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,NaN,True,NaN,False
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,NaN,True,NaN,False


In [34]:
events_merged.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929


In [62]:
test['prev_server_id'] = np.nan
test['prev_connected'] = np.nan

In [63]:
test.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected
3684102,2017-11-02 19:38:59.192,9529,168,16,False,16,2017-10-17 00:50:09.487556,2017-11-03 20:15:19.644,NaN,NaN
4729335,2017-11-26 22:13:32.274,40702,2359,20,False,20,2017-11-22 16:15:57.285482,2017-12-08 23:35:12.164,NaN,NaN
2401275,2017-10-04 15:22:18.894,9529,168,13,False,13,2017-09-26 16:10:36.230545,2017-10-10 20:31:43.062,NaN,NaN
7586288,2018-01-15 10:55:43.185,20685,1325,29,True,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974,NaN,NaN
9205922,2018-02-26 17:17:01.703,32193,3455,33,False,33,2018-02-26 15:52:13.678326,2018-03-01 18:05:54.051,NaN,NaN


In [56]:
test[test['device_id']==100].sort_values(by='timestamp')[['connected','server_id']]

,connected,server_id
8432,True,2
23740,False,2
24062,True,2
24779,True,2
32380,True,2
33307,False,2
33787,True,2
33850,True,2
35573,False,2


In [58]:
test[test['device_id']==100].sort_values(by='timestamp').groupby(by='device_id').shift(-1)[['connected','server_id']]

,connected,server_id
8432,False,2.0
23740,True,2.0
24062,True,2.0
24779,True,2.0
32380,False,2.0
33307,True,2.0
33787,True,2.0
33850,False,2.0
35573,NaN,NaN


In [70]:
test[test['device_id']==9529].sort_values(by='timestamp')

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected
307815,2017-08-02 17:51:12.058,9529,168,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,NaN,NaN
307921,2017-08-02 17:57:19.794,9529,168,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,NaN
307956,2017-08-02 17:58:20.248,9529,168,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,NaN
308358,2017-08-02 18:08:16.341,9529,168,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,NaN
308383,2017-08-02 18:09:29.2,9529,168,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
7898363,2018-01-20 13:56:37.807,9529,168,29,True,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974,29.0,NaN
7898488,2018-01-20 14:01:06.847,9529,168,29,True,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974,29.0,NaN
7898819,2018-01-20 14:11:20.483,9529,168,29,False,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974,29.0,NaN
7898871,2018-01-20 14:13:05.672,9529,168,29,True,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974,29.0,NaN


In [72]:
prev_server(test)
test.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected,prev_timestamp,ignored
3684102,2017-11-02 19:38:59.192,9529,168,16,False,16,2017-10-17 00:50:09.487556,2017-11-03 20:15:19.644,16.0,True,2017-11-02 19:52:51.83,False
4729335,2017-11-26 22:13:32.274,40702,2359,20,False,20,2017-11-22 16:15:57.285482,2017-12-08 23:35:12.164,20.0,True,2017-11-27 01:15:18.889,False
2401275,2017-10-04 15:22:18.894,9529,168,13,False,13,2017-09-26 16:10:36.230545,2017-10-10 20:31:43.062,13.0,False,2017-10-04 15:25:58.274,False
7586288,2018-01-15 10:55:43.185,20685,1325,29,True,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974,29.0,True,2018-01-15 11:11:30.299,False
9205922,2018-02-26 17:17:01.703,32193,3455,33,False,33,2018-02-26 15:52:13.678326,2018-03-01 18:05:54.051,33.0,True,2018-02-26 19:24:22.895,False


In [75]:
events_merged.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected,prev_timestamp,ignored
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-02 10:30:00,False
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-05-31 18:30:00,False
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-05-31 19:30:00,False
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-01 02:30:00,False
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-06 02:30:00,False


In [74]:
%%timeit
prev_server(events_merged)

27.6 s ± 901 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
def prev_connection(df):
    df['prev_server_id'] = df.sort(by='timestamp').groupby(by='device_id').shift(-1)['server_id']

In [25]:
%%timeit
pandarallel.initialize()
events_merged.parallel_apply(generate_previous_connection)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


TypeError: sort_values() got an unexpected keyword argument 'by'

In [15]:
events_merged[events_merged['prev_timestamp'].notna()].shape[0]

9251905

In [35]:
test = events_merged.sample(100000)
test.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at
3684102,2017-11-02 19:38:59.192,9529,168,16,False,16,2017-10-17 00:50:09.487556,2017-11-03 20:15:19.644
4729335,2017-11-26 22:13:32.274,40702,2359,20,False,20,2017-11-22 16:15:57.285482,2017-12-08 23:35:12.164
2401275,2017-10-04 15:22:18.894,9529,168,13,False,13,2017-09-26 16:10:36.230545,2017-10-10 20:31:43.062
7586288,2018-01-15 10:55:43.185,20685,1325,29,True,29,2018-01-12 17:01:54.553283,2018-02-12 15:06:52.974
9205922,2018-02-26 17:17:01.703,32193,3455,33,False,33,2018-02-26 15:52:13.678326,2018-03-01 18:05:54.051


In [69]:
test.dtypes

timestamp          object
device_id           int64
user_id             int64
server_id           int64
connected            bool
id                  int64
created_at         object
destroyed_at       object
prev_server_id    float64
prev_connected       bool
prev_timestamp     object
ignored              bool
dtype: object

In [97]:
test['ignored'].value_counts()

False    10000
Name: ignored, dtype: int64

In [103]:
# test['ignored'] = test.apply (lambda row: prepare_final(row), axis=1)
process_dataframe(test)

In [104]:
test['ignored'].value_counts()

False    9621
True      379
Name: ignored, dtype: int64

In [109]:
test[(test['connected']==False) & (test['prev_connected']==False)].shape[0]

4092

In [113]:
%%timeit
process_dataframe(events_merged)

2min 7s ± 3.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [134]:
%%timeit
pandarallel.initialize()
events_merged['ignored'] = events_merged.parallel_apply(lambda row: prepare_final(row), axis=1)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process

In [135]:
print(events_merged[(events_merged['connected']==False) & (events_merged['prev_connected']==False)].shape[0])
print(events_merged[events_merged['ignored']==True].shape[0])

379931
379931


In [123]:
df_split

[                       timestamp  device_id  user_id  server_id  connected  \
 1044783   2017-08-26 05:18:44.39      28100     1792          5       True   
 2906679   2017-10-17 00:13:52.05      24529     2233         15      False   
 9011778  2018-02-20 12:20:14.358      32164      119         32      False   
 327703   2017-08-03 07:53:05.195       9529      168          2      False   
 1588320  2017-09-10 00:45:53.564       9529      168         12       True   
 ...                          ...        ...      ...        ...        ...   
 2475145  2017-10-06 17:24:33.557       9529      168         13      False   
 604876   2017-08-09 23:29:57.733      14786      119          5      False   
 4344440  2017-11-18 11:10:05.964      11327      199         19       True   
 1027996  2017-08-25 16:34:03.722      24373     2422          5       True   
 5520827  2017-12-11 13:04:15.986      20685     1325         25       True   
 
          id                  created_at          

In [120]:
with Pool(4) as pool:
    results = pool.map_async(process_dataframe, test)
    l = results.get()

Process ForkPoolWorker-21:
Process ForkPoolWorker-22:
Process ForkPoolWorker-24:
Process ForkPoolWorker-23:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

In [ ]:


pandarallel.initialize()
test.parallel_apply(func, axis=1)

In [ ]:
events_merged['ignored']=events_merged.apply(prepare_final,axis=1)
print(events_merged.head())
events_merged['day'] = events_merged['timestamp'].dt.date
events_grouped = events_merged.groupby('day').count()
print(events_grouped)

In [ ]:
print("end:{}".format(datetime.datetime.now()))